# Ciência dos Dados - Projeto 03 - 

$Arthur\space Alegro \space de \space Oliveira$

$Pedro\space dos\space Santos \space e \space Silva$

$Jhonata\space Ferreira\space de \space Souza$

# 1. Introdução

## Importando Bibliotecas:

In [2]:
% matplotlib inline
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
import warnings
import random
import os

## Base de Dados:

Base de dados do ENEM referente aos anos de 2015 e 2016 extraídas pelo site http://portal.inep.gov.br/microdados.

Nomes dos arquivos:
* MICRODADOS_ENEM_2015.csv
* MICRODADOS_ENEM_2016.csv

**OBS.:** Para rodar corretamente este arquivo arquivo iPython Notebook (`.ipynb`)  deve-se extrair o arquivo `.zip` correspondente, acessar a pasta "DADOS" e colocar os arquivos `.csv` citados anteriormente no mesmo diretório deste arquivo.


## Diretório:

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\duals\Documents\GitHub\CDDP_final


#### Link para a base de dados já filtrada:
https://drive.google.com/open?id=1yq4DAJyJ2Er902X7Z1JuGc_t7w-aTIXJ

# 2. Minerando Dados e Características do Dataset

## Verificando dimensões dos arquivos:

In [3]:
def thousand_dot(number):
    """Convert int value in a string with dots as thousands separator"""
    lista = []
    count = 0
    for digit in (str(number))[::-1]:
        lista.append(digit)
        count +=1
        if count == 3:
            lista.append('.')
            count = 0
    lista.reverse
    elements = ''.join(lista)[::-1]
    return elements


def dimension_calculator(filename, chunksize=1000, sep=','):
    """Counts the amount of rows and columns on a .csv file"""
    # Raising warning:
    warnings.warn("\n\nWARNING! Depending on chunksize value and/or file size this process might take time to compute")
    # Parameters:
    rows = 0
    columns_labels = ''
    count = True
    # Process progress exibited to user:
    print("Processing...")
    # Opening database and counting values:
    for chunk in pd.read_csv(filename, chunksize=chunksize, encoding='latin-1', header=0, sep=sep):
        rows += len(chunk['NU_INSCRICAO'])
        if count == True:
            columns_labels = chunk.columns.values
            count = False
            
    # Process progress exibited to user:
    print("Process complete\n")   
    print("Number of rows: {}".format(thousand_dot(rows)))
    print("Number of columns: {}".format(len(columns_labels)))
    print("Number of elements: {}".format(thousand_dot(rows*len(columns_labels))))

* **2015**

In [4]:
dimension_calculator("MICRODADOS_ENEM_2015.csv")

C:\Users\duals\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: 

WARNING! Depending on chunksize value and/or file size this process might take time to compute


Processing...
Process complete

Number of rows: 7.746.427
Number of columns: 166
Number of elements: 1.285.906.882


* **2016**

In [5]:
dimension_calculator("MICRODADOS_ENEM_2016.csv", sep=';')

Processing...


C:\Users\duals\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: 

WARNING! Depending on chunksize value and/or file size this process might take time to compute


Process complete

Number of rows: 8.627.367
Number of columns: 166
Number of elements: 1.432.142.922


## Filtragem do Database:

Selecionamos uma parte da base de dados (100.000 rows de dados) para análise em escala reduzida.

Após todas as filtragens e cálculos forem concluídos, todo o processo será refeito para cada `chunk`, podendo aplicar as implementações para todo a base de dados muito mais rápido, sem a necessidade de carregar o arquivo completo de uma vez (o que pode ser impossível, pois, em alguns casos, demanda mais processamento e memória que o computador possui)

In [6]:
def save_df(to_open_filename, database_year,iteration_times, chunksize=1000, sep=','):
    # Raising warning:
    warnings.warn("\n\nWARNING! Depending on chunksize value and/or file size this process might take time to compute")

    # Counters and Status Controllers:
    overallcounter = 0
    chunkcounter = 0
    filecounter = 0
    shapecounter = [0, 0]
    to_save_filename = "test_dataframe_{}({}).csv".format(database_year, filecounter)
    # Process progress exibited to user:
    print("Opening file...")
    # Loading database in chunks and defining chunk size, correct enconding and reading configs:
    for chunk in pd.read_csv(to_open_filename, chunksize=chunksize, encoding='latin-1', header=0, sep=sep):

        # Selecting relevant parameters for dismiss useless data:
        chunk = chunk[(chunk["TP_PRESENCA_CN"] == 1) & (chunk["TP_PRESENCA_CH"] == 1) & 
                      (chunk["TP_PRESENCA_LC"] == 1) & (chunk["TP_PRESENCA_MT"] == 1) & 
                      (chunk["TP_STATUS_REDACAO"] == 1) & (chunk["IN_TREINEIRO"] == 0)]

        # Selecting relevant parameters for fitering data:
        chunk = chunk.loc[:, ["SG_UF_RESIDENCIA", "NU_IDADE", "TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA", "TP_ST_CONCLUSAO", 
                              "TP_ESCOLA", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO", "Q006"]]

        # Exporting dataframe to new .csv file below existent data:
        chunk.to_csv(to_save_filename, index=False)

        # Counters update and process progress exibited to user:
        chunkcounter += 1
        overallcounter += 1
        filecounter += 1
        shapecounter[0] += chunk.shape[0]
        shapecounter[1] = chunk.shape[1]
        print("Processing... ({}/{}) - {} rows processed - Saving in '{}')".format(overallcounter, iteration_times, shapecounter[0], to_save_filename))
        to_save_filename = "test_dataframe_{}({}).csv".format(database_year, filecounter)
    print("Process Completed")

* **2015**

In [7]:
# Creating filtered .csv file:
save_df('MICRODADOS_ENEM_2015.csv', database_year=2015, chunksize=1000000, iteration_times=8)

C:\Users\duals\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: 

WARNING! Depending on chunksize value and/or file size this process might take time to compute
  This is separate from the ipykernel package so we can avoid doing imports until


Opening file...
Processing... (1/8) - 723612 rows processed - Saving in 'test_dataframe_2015(0).csv')
Processing... (2/8) - 1425321 rows processed - Saving in 'test_dataframe_2015(1).csv')
Processing... (3/8) - 2097387 rows processed - Saving in 'test_dataframe_2015(2).csv')
Processing... (4/8) - 2742262 rows processed - Saving in 'test_dataframe_2015(3).csv')
Processing... (5/8) - 3346414 rows processed - Saving in 'test_dataframe_2015(4).csv')
Processing... (6/8) - 3928083 rows processed - Saving in 'test_dataframe_2015(5).csv')
Processing... (7/8) - 4429373 rows processed - Saving in 'test_dataframe_2015(6).csv')
Processing... (8/8) - 4768463 rows processed - Saving in 'test_dataframe_2015(7).csv')
Process Completed


* **2016**

In [8]:
# Creating filtered .csv file:
save_df('MICRODADOS_ENEM_2016.csv', database_year=2016, chunksize=1000000, iteration_times=9, sep=';')

Opening file...


C:\Users\duals\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: 

WARNING! Depending on chunksize value and/or file size this process might take time to compute
  This is separate from the ipykernel package so we can avoid doing imports until


Processing... (1/9) - 658887 rows processed - Saving in 'test_dataframe_2016(0).csv')
Processing... (2/9) - 1309576 rows processed - Saving in 'test_dataframe_2016(1).csv')
Processing... (3/9) - 1932155 rows processed - Saving in 'test_dataframe_2016(2).csv')
Processing... (4/9) - 2540238 rows processed - Saving in 'test_dataframe_2016(3).csv')
Processing... (5/9) - 3116716 rows processed - Saving in 'test_dataframe_2016(4).csv')
Processing... (6/9) - 3656028 rows processed - Saving in 'test_dataframe_2016(5).csv')
Processing... (7/9) - 4154448 rows processed - Saving in 'test_dataframe_2016(6).csv')
Processing... (8/9) - 4604923 rows processed - Saving in 'test_dataframe_2016(7).csv')
Processing... (9/9) - 4868906 rows processed - Saving in 'test_dataframe_2016(8).csv')
Process Completed


## Verificando integridade dos arquivos:

In [5]:
filename_list_2015 = ["test_dataframe_2015(0).csv", "test_dataframe_2015(1).csv", 
                      "test_dataframe_2015(2).csv", "test_dataframe_2015(3).csv", 
                      "test_dataframe_2015(4).csv", "test_dataframe_2015(5).csv", 
                      "test_dataframe_2015(6).csv", "test_dataframe_2015(7).csv"]

filename_list_2016 = ["test_dataframe_2016(0).csv", "test_dataframe_2016(1).csv",
                      "test_dataframe_2016(2).csv", "test_dataframe_2016(3).csv", 
                      "test_dataframe_2016(4).csv", "test_dataframe_2016(5).csv", 
                      "test_dataframe_2016(6).csv", "test_dataframe_2016(7).csv", 
                      "test_dataframe_2016(8).csv"]

In [6]:
def integrity_verification(filename_list, database_year):
    """Counts the amount of rows in multiple .csv files"""
    rows_count = 0
    for filename in filename_list:
        for chunk in pd.read_csv(filename, chunksize=1000, encoding='latin-1', header=0, sep=','):
            rows_count += chunk.shape[0]
    print("{} database - Total rows: {}".format(database_year, rows_count))

In [8]:
integrity_verification(filename_list_2015, 2015)
integrity_verification(filename_list_2016, 2016)

2015 database - Total rows: 4768463
2016 database - Total rows: 4868906


## Separando Bases (teste e treinamento):

In [45]:
def database_separation(filename_list, database_year, sep=',', chunksize=1000):
    """Generates random sample for training and test databases and exports to separated .csv files"""
    # Raising warning:
    warnings.warn("\n\nWARNING! Depending on chunksize value, files amount and/or files size this process might take time to compute")
    # Dataframes to recieve chunks' data:
    training_df = pd.DataFrame(data=None)
    test_df = pd.DataFrame(data=None)
    # Counter:
    chunkcounter = 0
    # Process progress exibited to user:
    print("Process iniciated...")
    # Reading all data from filtered .csv files:
    for filename in filename_list:
        for chunk in pd.read_csv(filename, chunksize=chunksize, encoding='latin-1', header=0, sep=sep):
            # Creating list with chunk indexers:
            indexers = list(chunk.index)
            
            # Random shuffling chunk's index values:
            random.shuffle(indexers)
            
            # Separating 80% to training database and 20% to test database:
            training_chunk_idx = indexers[:int((chunkcounter + 0.8)*chunksize)]
            test_chunk_idx = indexers[int((chunkcounter + 0.8)*chunksize):]

            # Sorting chunk indexers:
            training_chunk_idx.sort()
            test_chunk_idx.sort()

            # Slicing chunk's data:
            training_chunk = chunk.iloc[training_chunk_idx, :]
            test_chunk = chunk.iloc[test_chunk_idx, :]
            print(training_chunk_idx)
            # Setting 'is_copy' argument (avoid file corruption):
            training_chunk.is_copy
            test_chunk.is_copy
            
            # Appending separated chunk data to reference DataFrame:
            pd.concat([training_df, training_chunk], ignore_index=True)
            pd.concat([training_df, test_chunk], ignore_index=True)
        
            # Counters update and process progress exibited to user:
            chunkcounter += 1
            print("{} chunks processed - {} rows processed".format(chunkcounter, chunkcounter*chunksize))
    # Process progress exibited to user:
    print("Exporting data...")
    
    # Exporting DataFrames to separated .csv files:
    training_df.to_csv("training_database_{}.csv".format(database_year), index=False)
    test_df.to_csv("test_database{}.csv".format(database_year), index=False)
    
    # Process progress exibited to user:
    print("Process complete")

* **2015**

In [46]:
database_separation(filename_list_2015, 2015, chunksize=10000)

Process iniciated...
[0, 1, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 26, 27, 28, 29, 30, 31, 32, 34, 35, 37, 38, 41, 42, 43, 44, 45, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 86, 87, 88, 90, 91, 92, 96, 97, 98, 99, 100, 101, 102, 104, 105, 107, 108, 109, 110, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 161, 162, 163, 164, 165, 167, 168, 169, 170, 171, 172, 175, 176, 178, 179, 181, 182, 183, 184, 185, 187, 188, 190, 191, 193, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 220, 221, 223, 224, 225, 226, 228, 229, 230, 231, 232, 234, 235, 236, 237, 238, 239, 242, 243, 245, 246, 247, 248, 251, 252, 253, 256, 258, 259, 260, 261, 263, 265, 267, 268, 269, 270, 271

C:\Users\duals\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: 

WARNING! Depending on chunksize value, files amount and/or files size this process might take time to compute
  after removing the cwd from sys.path.


IndexError: positional indexers are out-of-bounds

* **2016**

In [22]:
database_separation(filename_list_2016, 2016, chunksize=10000)

IndexError: positional indexers are out-of-bounds

In [15]:
help(pd.concat)

Help on function concat in module pandas.core.reshape.concat:

concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
    Concatenate pandas objects along a particular axis with optional set logic
    along the other axes.
    
    Can also add a layer of hierarchical indexing on the concatenation axis,
    which may be useful if the labels are the same (or overlapping) on
    the passed axis number.
    
    Parameters
    ----------
    objs : a sequence or mapping of Series, DataFrame, or Panel objects
        If a dict is passed, the sorted keys will be used as the `keys`
        argument, unless it is passed, in which case the values will be
        selected (see below). Any None objects will be dropped silently unless
        they are all None in which case a ValueError will be raised
    axis : {0/'index', 1/'columns'}, default 0
        The axis to concatenate along
    join : {'inner', 'oute

In [4]:
help(pd.DataFrame.append)

Help on function append in module pandas.core.frame:

append(self, other, ignore_index=False, verify_integrity=False)
    Append rows of `other` to the end of this frame, returning a new
    object. Columns not in this frame are added as new columns.
    
    Parameters
    ----------
    other : DataFrame or Series/dict-like object, or list of these
        The data to append.
    ignore_index : boolean, default False
        If True, do not use the index labels.
    verify_integrity : boolean, default False
        If True, raise ValueError on creating index with duplicates.
    
    Returns
    -------
    appended : DataFrame
    
    Notes
    -----
    If a list of dict/series is passed and the keys are all contained in
    the DataFrame's index, the order of the columns in the resulting
    DataFrame will be unchanged.
    
    Iteratively appending rows to a DataFrame can be more computationally
    intensive than a single concatenate. A better solution is to append
    those r

Nome das colunas:
* `SG_UF_RESIDENCIA`: Sigla da Unidade de Federação da residência do candidato
* `NU_IDADE`: Idade do candidato
* `TP_SEXO`: Sexo do candidato
* `TP_ESTADO_CIIVL`: Estado Civil do candidato
* `TP_COR_RACA`: Grupo étinco a qual o candidato se identifica
* `TP_ST_CONCLUSAO`: Situação de conclusão do Ensino Médio
* `TP_ESCOLA`: Tipo de escola do Ensino Médio
* `NU_NOTA`: Notas nas avaliações do ENEM, sendo:

|Sigla   |    Descrição        |
|--------|---------------------|
|CN      |Ciências da Natureza |
|CH      |Ciências Humanas     |
|LC      |Linguagens e Códigos |
|MT      |Matemática           |
|REDACAO |Redação              |
    
* `Q006`: Renda mensal da família

# Porquê essas colunas foram escolhidas?

- A Unidade de Federação é bem relevante no acesso à escolaridade, pois sabe-se que, no Brasil, educação de boa qualidade, de maneira geral, é acessada por meios privados. Sendo assim, há regiões que são pobres em desenvolvimento humano.
- Idade influencia, tendo em vista que um jovem de 17-20 anos tende a lembrar mais do conteudo ante a um adulto de 30+
- Grupo Étnico (vulgarmente conhecido como "Raça") tem grande peso na nota, tendo em vista o sistema de cotas
- Situação de conclusão do Ensino Médio diz, aproximadamente, se o candidato se encontra preparado para a prova, ou não
- Tipo de escola nos ajuda a montar o perfil do aluno, e  o que esperar do seu desempenho
- Nota da prova é útil para a análise pra estabelecer a base de treinamento

# Análise descritiva:

- Gráficos de como cada variável afeta na nota do candidato
    - Boxplot de renda com nota
    - Gráfico de barras para nota com outra variável qualitativa
    

## Bibliografia:

Sites consultados:
* https://openpyxl.readthedocs.io/en/2.5/pandas.html
* http://portal.inep.gov.br/microdados
* 
* 
* 
* 
* 
* 